In [1]:
from numba import cuda
import numpy as np
import time
print(cuda.gpus)

<Managed Device 0>


# Реализация алгоритма градиентного спуска на процессоре

### Функция Розенброка $((1 - x) ^ 2 + 100 * (y - x * x) ^ 2 )$

In [2]:
def f1(x):
    return ((1 - x[0]) ** 2 + 100 * (x[1] - x[0] * x[0]) ** 2 )
def df1(x):
    return np.array([2 * x[0] - 2 + 400 * x[0] * (x[0] * x[0] - x[1]),
                      200 * (x[1] - x[0] * x[0])])

In [3]:
def my_gd(X0, lr = 0.001, N_it = 3000000):
    model = X0.copy()
    for i in range(N_it):
        model -= lr * df1(model)
    return model

In [4]:
X0 = np.array([2.0,2.0])

start_time = time.time()
param = my_gd(X0, lr = 0.001)
time1 = time.time() - start_time

print("Время работы %s секунд" % (time1))
print("Начальная точка", X0, "\t\t\tзначение функции в ней", f1(X0))
print("Найденный оптимум",param, "\t\t\tзначение функции в нем",f1(param))
print("Теоритический оптимум",[1,1], "\t\t\tзначение функции в нем",f1([1,1]))
print("Разница x",abs(param - [1,1]), "\tf(x)",abs(f1(param) -f1([1,1])))

Время работы 10.135204792022705 секунд
Начальная точка [2. 2.] 			значение функции в ней 401.0
Найденный оптимум [1. 1.] 			значение функции в нем 2.0850579801122868e-26
Теоритический оптимум [1, 1] 			значение функции в нем 0
Разница x [1.44328993e-13 2.89102076e-13] 	f(x) 2.0850579801122868e-26


# Реализация алгоритма градиентного спуска на cuda с помощью numba

In [5]:
@cuda.jit
def my_gd_cuda(X0, lr, N_it):
    tx = cuda.threadIdx.x
    for i in range(N_it):
        if tx == 0:
            X0[tx] -= lr * (2.0 * X0[0] - 2.0 + 400.0 * X0[0] * (X0[0] * X0[0] - X0[1]))
        else:
            X0[tx] -= lr * (200.0 * (X0[1] - X0[0] * X0[0]))

In [6]:
import numpy
X0 = np.array([2.0,2.0])
lr = 0.001
N_it = 3000000

start_time = time.time()
my_gd_cuda[1, 2](X0, lr,N_it)
time2 = time.time() - start_time

print("Время работы %s секунд" % (time2))
print("Начальная точка", [2.0,2.0], "\t\t\tзначение функции в ней", f1([2.0,2.0]))
print("Найденный оптимум",X0, "\t\t\tзначение функции в нем",f1(X0))
print("Теоритический оптимум",[1,1], "\t\t\tзначение функции в нем",f1([1,1]))
print("Разница x",abs(X0 - [1,1]), "\tf(x)",abs(f1(param) -f1([1,1])))

d:\python39\lib\site-packages\numba\cuda\dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\python39\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Время работы 1.745039701461792 секунд
Начальная точка [2.0, 2.0] 			значение функции в ней 401.0
Найденный оптимум [1. 1.] 			значение функции в нем 2.0786534156380237e-26
Теоритический оптимум [1, 1] 			значение функции в нем 0
Разница x [1.44106949e-13 2.88657986e-13] 	f(x) 2.0850579801122868e-26


In [7]:
print("Достигнуто ускорение в {:0.3f} раз".format(time1/time2))

Достигнуто ускорение в 5.808 раз
